In [13]:
# Connect to db using dataset:
import dataset
import datafreeze
db = dataset.connect('mssql+pyodbc://ORLEBIDEVDB/INTEGRATION?driver=SQL+Server+Native+Client+11.0')

# print(db.tables)

In [14]:
# Grab the mktcollection table:
mktCollect_Table = db['INT_MKTCollectionDetails']
print(mktCollect_Table.columns)

dimdate_Table = db['VW_INT_DIMDATE']

# print(dimdate_Table.columns
list(mktCollect_Table.find(_limit=10))

['CollectionDetailSK', 'DonationTypeSK', 'CollectionDateSK', 'LocationSK', 'motivationsk', 'SourceSystem', 'registrationID', 'personid', 'driveid', 'failurecode', 'CompletedFlag', 'UnitNumber', 'DriveName', 'createddate', 'modifieddate', 'bloodtypesk', 'discarded', 'import', 'EthnicSK', 'Gender', 'vehicleUID', 'vehicleId', 'MachineID', 'PrePhlebEmpid', 'PostPhlebEmpid', 'RegEmpid', 'PersonDOB', 'PersonZipCode', 'LoadFlag', 'DHQEmpID', 'PhyEmpID', 'ProductCount', 'StagingAreaSK', 'MachineSerialNo', 'TeamLeadEmpID', 'GroupID', 'AccountID', 'AccountInternalName', 'AccountZipCode', 'DrawTime', 'WithdrawlTime', 'ParentVolume']


[OrderedDict([('CollectionDetailSK', 3439560),
              ('DonationTypeSK', 1),
              ('CollectionDateSK', 20180604),
              ('LocationSK', 125),
              ('motivationsk', 2),
              ('SourceSystem', 'RSA'),
              ('registrationID', 108451707),
              ('personid', 53021103),
              ('driveid', 2170872),
              ('failurecode', None),
              ('CompletedFlag', '12'),
              ('UnitNumber', 'W036818597255'),
              ('DriveName', 'R3-Vero Beach Mobiles'),
              ('createddate',
               datetime.datetime(2018, 6, 5, 6, 40, 33, 263000)),
              ('modifieddate',
               datetime.datetime(2018, 8, 3, 6, 40, 36, 927000)),
              ('bloodtypesk', 1),
              ('discarded', None),
              ('import', 0),
              ('EthnicSK', 7),
              ('Gender', 'F'),
              ('vehicleUID', 1),
              ('vehicleId', 772),
              ('MachineID', '12'),
          

In [ ]:
# Querying:
mktJul2018Donors = db.query(
"""
select
    mkt.person_id,
    dd.FullDateUSA,
    mkt.min_reg_id,
    mkt.total_platelet_donations
from
(
    select MKT.personid PERSON_ID, MIN(mkt.COLLECTIONDATESK) MIN_REG_DATE,MIN(registrationid) MIN_REG_ID,COUNT(1) total_platelet_donations
    from [Integration].[dbo].[INT_MKTCollectionDetails] mkt
    where mkt.DonationTypeSK in (2,5,7,26)		
     and mkt.CompletedFlag >= 8		
     and upper(mkt.GENDER) in ('M','F')		
     and (MKT.collectiondatesk >= '20180701' and MKT.collectiondatesk < '20180801')
    group by MKT.personid
) mkt
JOIN [Integration].[dbo].[VW_INT_DIMDATE] dd
    ON dd.dateKey = mkt.MIN_REG_DATE
""")

# for row in mktJul2018Donors:
#   print(row['PERSON_ID'], row['total_platelet_donations'])


# Output to CSV:
#datafreeze.freeze(mktJul2018Donors, format='csv',
#                  filename='data/mktCollect_Jul18_platelet_dons.csv')
                            
# Output to JSON:
datafreeze.freeze(mktJul2018Donors, format='json', wrap=False,
                  filename='data/mktCollect_Jul18_plDons.json')